In [ ]:
+--------------------------+
|                          |
|                          |
|           Model          |
|                          |
|                          |
+--------------------------+
             ^ 
             |
             |
+--------------------------+                         +--------------------------+
|                          |                         |                          |
|                          |                         |                          |
|        Dataloaders       |                         |   External Data Users    |
|                          |                         |                          |
|                          |                         |                          |
+--------------------------+                         +--------------------------+
               ^ `~.                                         .~^ ^
                      `¬.                               .¬`
                            ¬-                     -¬`
                                 "~.         ,-^
                           +--------------------------+
                           |                          |
                           |                          |
                           |     Geographic Index     |
                           |                          |
                           |                          |
                           +--------------------------+
                           .~¬`^                  ^`¬-
                    ,-¬^                                `─.
+--------------------------+                       +--------------------------+
|                          |                       |                          |
|                          |                       |                          |
|   Shapefiles, Geotiffs   |                       |         Metadata         |
|                          |                       |                          |
|                          |                       |                          |
+--------------------------+                       +--------------------------+

In [1]:
import sys
sys.path = ["../../"] + sys.path
import importlib
import os
from fs import open_fs

# Import the factory that produces our map data structure for a given geographic region
from src.data.index.map_data_factory import MapDataFactory

In [2]:
# Copy the geographic index to the local directory
!gsutil cp gs://ml4cc_data_lake/0_DEV/2_Mart/0_InformationMart/geographic_index.pkl .

Copying gs://ml4cc_data_lake/0_DEV/2_Mart/0_InformationMart/geographic_index.pkl...
/ [1 files][986.9 KiB/986.9 KiB]                                                
Operation completed over 1 objects/986.9 KiB.                                    


In [3]:
# Instantiate a MapDataFactory from a saved geographic index structure
data_factory = MapDataFactory("geographic_index.pkl")
my_map_data = data_factory.create_map_data(37, 12, 38, 13)

In [4]:
# We see the contents of a dataclass corresponding to the selected map region, let's explore...
print(my_map_data)

MapData(metadata={MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955)), MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_02PORTOPALO_DEL_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955)), MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955)), MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_01RATTALORO_DEL_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955))}, satellite_images={'S2': {MapDataSource(path='worldfloods/tiffimages/S2/EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/S2/EMSR333_02PORTOPALO_DEL_v1_observed_event_a.tif', last_modified=datetime

In [5]:
# That MapData readout looks quite crowded-- let's pick it apart
help(my_map_data)

Help on MapData in module src.data.index.map_data object:

class MapData(builtins.object)
 |  Represents known data for a subset of the map separated by type of data
 |  
 |  @field metadata Set of MapDataSources with JSON metadata
 |  @field satellite_images Dictionary of Sets of MapDataSources with satellite image GeoTIF data, one set per data source
 |  @field floodmaps Set of MapDataSources containing shape files that detail flood-related geographic information
 |  @field cloudmasks Set of MapDataSources with GeoTIF cloud masks
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |  
 |  __init__(self, min_lat, min_lon, max_lat, max_lon, index_list)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak refer

In [6]:
# We see a few JSON files as the available metadata for resources in our selected map region
print(my_map_data.metadata)

{MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955)), MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_02PORTOPALO_DEL_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955)), MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955)), MapDataSource(path='worldfloods/tiffimages/meta/EMSR333_01RATTALORO_DEL_v1_observed_event_a.json', last_modified=datetime.datetime(2021, 2, 26, 20, 49, 18, 703955))}


In [7]:
# We see that there is satellite image data available from Landsat-8, Sentinel-2, and gt in the map region
print(my_map_data.satellite_images)

{'S2': {MapDataSource(path='worldfloods/tiffimages/S2/EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/S2/EMSR333_02PORTOPALO_DEL_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/S2/EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/S2/EMSR333_01RATTALORO_DEL_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52))}, 'L8': {MapDataSource(path='worldfloods/tiffimages/L8/EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/L8/EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/L8/EMSR33

In [8]:
# We see that there are multiple floodmap shape files available in the selected map region
print(my_map_data.floodmaps)

{MapDataSource(path='worldfloods/tiffimages/floodmaps/EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a.shp', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/floodmaps/EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a.shp', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/floodmaps/EMSR333_02PORTOPALO_DEL_v1_observed_event_a.shp', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/floodmaps/EMSR333_01RATTALORO_DEL_v1_observed_event_a.shp', last_modified=datetime.datetime(2018, 11, 4, 9, 52))}


In [9]:
# We see that there are multiple cloud masks available in the selected map region
print(my_map_data.cloudmasks)

{MapDataSource(path='worldfloods/tiffimages/cloudprob/EMSR333_01RATTALORO_DEL_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/cloudprob/EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/cloudprob/EMSR333_02PORTOPALO_DEL_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52)), MapDataSource(path='worldfloods/tiffimages/cloudprob/EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a.tif', last_modified=datetime.datetime(2018, 11, 4, 9, 52))}


In [10]:
# Get and display a geotiff
with open_fs("gs://ml4floods/" + list(my_map_data.satellite_images["S2"])[0].path + "?strict=False") as fs:
    with fs.open('', 'rb') as geotiff_file:
        print(geotiff_file.read(100))

b'II*\x00\xc0\x00\x00\x00GDAL_STRUCTURAL_METADATA_SIZE=000140 bytes\nLAYOUT=IFDS_BEFORE_DATA\nBLOCK_ORDER=ROW_MAJOR\nBLO'
